## Following code reads crunchbase web link for each startup from 'startup.csv' and fetches the startup's information from crunchbase.com. It then writes the info in 'startup2.csv'. Because crunchbase.com does not allow webscraping we cannot hit requests continuously. So a random delay is added after every request.

## Even after putting the delay, requests get blocked after some time. In this case if we run the main block again then the data gathering starts after our last successfully attempt. That is the program checks which startups' data has already been collected and starts hitting requests for startups after that point.

In [0]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import csv
import random
from urllib.request import urlopen, Request

In [0]:
def getSpanTagData(soup,fieldName):
    span_tags = soup.findAll("span")
    i=0
    found = False
    while i < len(span_tags):
        if (span_tags[i].get_text() == fieldName):
            #print(span_tags[i])
            found = True
            break
        i+=1
    i+=3
    if found == True:
        return span_tags[i].get_text()
    else:
        return ""

In [0]:
def findAlreadyFoundCount():
    with open('D:\\cmpe256\\Team project\\startups2.csv','r') as file1:
        csvreader = csv.reader(file1)

        row = csvreader.__next__()
        already_found_count = sum(1 for row in csvreader)
        #print(already_found_count)
    file1.close()
    return already_found_count

### Main block to run for getting cruchbase data

In [0]:
with open('D:\\cmpe256\\Team project\\startups.csv','r') as file1:
    csvreader = csv.reader(file1)

    row = csvreader.__next__()
    skip_count = findAlreadyFoundCount()
    while skip_count > 0:
        row = csvreader.__next__()
        skip_count-=1

    count = 1
    useragent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    headers = {'User-Agent' : useragent}

    with open('D:\\cmpe256\\Team project\\startups2.csv','a',newline="") as file2:
        writer = csv.writer(file2)
        """writer.writerow(['Name','State','Crunchbase link','Angel Link','Website','Start date','Exit Value',
                             'Unknown','Funding','Categories','Region','IPO Status','Last Funding Type',
                             'Number of Funding Rounds','Number of Articles','Number of Employees'])"""


        for row in csvreader:
            print (row[0])
            if 'crunchbase' in row[2]:
                url=row[2]
                req = urllib.request.Request(url,headers=headers)
                with urllib.request.urlopen(req) as response:
                    startup_page = response.read()
                    soup = BeautifulSoup(startup_page, "html.parser")
                    span_tags = soup.findAll("span")

                    categories = getSpanTagData(soup,"Categories")
                    region = getSpanTagData(soup,"Headquarters Regions")
                    ipostatus = getSpanTagData(soup,"IPO Status")
                    lastfundtype = getSpanTagData(soup,"Last Funding Type")
                    fundingRounds = getSpanTagData(soup,"Number of Funding Rounds")
                    articles = getSpanTagData(soup,"Number of Articles")
                    employees = getSpanTagData(soup,"Number of Employees")
                    employees = " " + employees
                    investors_count = getSpanTagData(soup,"Number of Investors")
                    info = [categories,region,ipostatus,lastfundtype,fundingRounds,articles,employees,investors_count] 
                    print([categories,region,ipostatus,lastfundtype,fundingRounds,articles,employees,investors_count])

                    for field in info:
                        row.append(field)
            writer.writerow(row)
            time.sleep(random.randint(41,94))